**Fine-Tuning of BERT Models for Question Answering**

This notebook uses pretrained Hugging Face BERT models and finetunes the models for question answering tasks. This notebook uses example code provided by Hugging Face for finetuning a model for question answering [[Hugging Face Question Answering]](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/question_answering.ipynb#scrollTo=HFASsisvIrIb). The code has been modified to finetune on the SQuAD 2.0 and Google NQ datasets.

In [1]:
!pip install datasets transformers

     |████████████████████████████████| 194kB 5.8MB/s 
     |████████████████████████████████| 2.0MB 34.6MB/s 
     |████████████████████████████████| 112kB 40.6MB/s 
     |████████████████████████████████| 245kB 37.8MB/s 
     |████████████████████████████████| 3.3MB 26.3MB/s 
     |████████████████████████████████| 870kB 45.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=d75929abaf37fae59f3fbc53628a02225acb9f26341f03d1d5108064c271c917
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [2]:
########### INPUT ###########
# Input the base model, batch size, and whether or not the dataset contains non-answerable questions (squad_v2 = True)
model_checkpoint = "bert-base-uncased"
batch_size = 16
squad_v2 = True

Load Data

In [3]:
# import the load_dataset and load_metric for loading and evaluation of datasets
from datasets import load_dataset, load_metric

In [4]:
########### INPUT ###########
# Load the dataset to finetune model
import json
import pandas as pd
#datasets = load_dataset('json', data_files='/content/drive/MyDrive/ColabNotebooks/data/nq_train.jsonl')
datasets = load_dataset('json', data_files={'train': '/content/drive/MyDrive/ColabNotebooks/data/nq-cl-reduce_train.jsonl',
                                              'validation': '/content/drive/MyDrive/ColabNotebooks/data/nq-cl-reduce_validation.jsonl'})

Using custom data configuration default-ad5edf2efffd1c1c


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-ad5edf2efffd1c1c/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

In [6]:
# view dataset object
datasets

DatasetDict({
    train: Dataset({
        features: ['answers', 'context', 'id', 'question'],
        num_rows: 7856
    })
    validation: Dataset({
        features: ['answers', 'context', 'id', 'question'],
        num_rows: 1964
    })
})

In [7]:
# view example from dataset
datasets["train"][0]

{'answers': {'answer_start': [4311], 'text': ['late twentieth century']},
 'context': " Automobile air conditioning ( also called A / C ) systems use air conditioning to cool the air in a vehicle .       A company in New York City in the United States first offered installation of air conditioning for cars in 1933 . Most of their customers operated limousines and luxury cars .   In 1939 , Packard became the first automobile manufacturer to offer an air conditioning unit in its cars . These were manufactured by Bishop and Babcock Co , of Cleveland , Ohio . The `` Bishop and Babcock Weather Conditioner '' also incorporated a heater . Cars ordered with the new `` Weather Conditioner '' were shipped from Packard 's East Grand Boulevard facility to the B&B factory where the conversion was performed . Once complete , the car was shipped to a local dealer where the customer would take delivery .   Packard fully warranted and supported this conversion , and marketed it well . However , it was 

In [8]:
# code to view random examples in pandas
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=3):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(datasets["train"])

Preprocess Training Data

In [10]:
# import the correct tokenizer for the model architecture
from transformers import AutoTokenizer    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [11]:
# verify that the tokenizer is a fast tokenizer
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [12]:
# run a check to verify that the tokenizer is working for an example questions and answer
tokenizer("Is this tokenizer working?", "Yes, the tokenizer is working correctly")

{'input_ids': [101, 2003, 2023, 19204, 17629, 2551, 1029, 102, 2748, 1010, 1996, 19204, 17629, 2003, 2551, 11178, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
########### INPUT ###########
# Length will be truncated to handle long contexts
# Set the max length (questions and context) and stride (context overlap)
max_length = 384
doc_stride = 128 

In [14]:
# verify that the truncation is working correctly by finding an example
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = datasets["train"][i]

Token indices sequence length is longer than the specified maximum sequence length for this model (1043 > 512). Running this sequence through the model will result in indexing errors


In [15]:
# check to see what the length of the example is without truncation (should be greater than max_length)
len(tokenizer(example["question"], example["context"])["input_ids"])

1043

In [16]:
# tokenizer should return always return the question plus truncated contexts
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

In [17]:
# verify the length for the mulitple examples are provided for tokenized example
[len(x) for x in tokenized_example["input_ids"]]

[384, 384, 384, 308]

In [18]:
# decode the tokenized example to verify that we have a question plus the truncated context
for x in tokenized_example["input_ids"][:]:
    print(tokenizer.decode(x))

[CLS] when did air conditioning became standard in cars [SEP] automobile air conditioning ( also called a / c ) systems use air conditioning to cool the air in a vehicle. a company in new york city in the united states first offered installation of air conditioning for cars in 1933. most of their customers operated limousines and luxury cars. in 1939, packard became the first automobile manufacturer to offer an air conditioning unit in its cars. these were manufactured by bishop and babcock co, of cleveland, ohio. the ` ` bishop and babcock weather conditioner'' also incorporated a heater. cars ordered with the new ` ` weather conditioner'' were shipped from packard's east grand boulevard facility to the b & b factory where the conversion was performed. once complete, the car was shipped to a local dealer where the customer would take delivery. packard fully warranted and supported this conversion, and marketed it well. however, it was not commercially successful for a number of reason

In [19]:
# use the tokenizer to map the offset for locating the answer
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 4), (5, 8), (9, 12), (13, 25), (26, 32), (33, 41), (42, 44), (45, 49), (0, 0), (1, 11), (12, 15), (16, 28), (29, 30), (31, 35), (36, 42), (43, 44), (45, 46), (47, 48), (49, 50), (51, 58), (59, 62), (63, 66), (67, 79), (80, 82), (83, 87), (88, 91), (92, 95), (96, 98), (99, 100), (101, 108), (109, 110), (117, 118), (119, 126), (127, 129), (130, 133), (134, 138), (139, 143), (144, 146), (147, 150), (151, 157), (158, 164), (165, 170), (171, 178), (179, 191), (192, 194), (195, 198), (199, 211), (212, 215), (216, 220), (221, 223), (224, 228), (229, 230), (231, 235), (236, 238), (239, 244), (245, 254), (255, 263), (264, 273), (273, 274), (275, 278), (279, 285), (286, 290), (291, 292), (295, 297), (298, 302), (303, 304), (305, 312), (313, 319), (320, 323), (324, 329), (330, 340), (341, 353), (354, 356), (357, 362), (363, 365), (366, 369), (370, 382), (383, 387), (388, 390), (391, 394), (395, 399), (400, 401), (402, 407), (408, 412), (413, 425), (426, 428), (429, 435), (436, 439), 

In [20]:
# verify that the mapping is working correctly
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

when when


In [21]:
# use the tokenizer to find sequence ids for locating the position of the question and answer
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [22]:
# answers = example["answers"]
# start_char = answers["answer_start"]
# start_char
example

{'answers': {'answer_start': [4311], 'text': ['late twentieth century']},
 'context': " Automobile air conditioning ( also called A / C ) systems use air conditioning to cool the air in a vehicle .       A company in New York City in the United States first offered installation of air conditioning for cars in 1933 . Most of their customers operated limousines and luxury cars .   In 1939 , Packard became the first automobile manufacturer to offer an air conditioning unit in its cars . These were manufactured by Bishop and Babcock Co , of Cleveland , Ohio . The `` Bishop and Babcock Weather Conditioner '' also incorporated a heater . Cars ordered with the new `` Weather Conditioner '' were shipped from Packard 's East Grand Boulevard facility to the B&B factory where the conversion was performed . Once complete , the car was shipped to a local dealer where the customer would take delivery .   Packard fully warranted and supported this conversion , and marketed it well . However , it was 

In [23]:
list(map(lambda i:i, example["context"]))[4311:]

['l',
 'a',
 't',
 'e',
 ' ',
 't',
 'w',
 'e',
 'n',
 't',
 'i',
 'e',
 't',
 'h',
 ' ',
 'c',
 'e',
 'n',
 't',
 'u',
 'r',
 'y',
 ' ',
 '.',
 ' ',
 'A',
 'l',
 't',
 'h',
 'o',
 'u',
 'g',
 'h',
 ' ',
 'a',
 'i',
 'r',
 ' ',
 'c',
 'o',
 'n',
 'd',
 'i',
 't',
 'i',
 'o',
 'n',
 'e',
 'r',
 's',
 ' ',
 'u',
 's',
 'e',
 ' ',
 's',
 'i',
 'g',
 'n',
 'i',
 'f',
 'i',
 'c',
 'a',
 'n',
 't',
 ' ',
 'p',
 'o',
 'w',
 'e',
 'r',
 ' ',
 ';',
 ' ',
 't',
 'h',
 'e',
 ' ',
 'd',
 'r',
 'a',
 'g',
 ' ',
 'o',
 'f',
 ' ',
 'a',
 ' ',
 'c',
 'a',
 'r',
 ' ',
 'w',
 'i',
 't',
 'h',
 ' ',
 'c',
 'l',
 'o',
 's',
 'e',
 'd',
 ' ',
 'w',
 'i',
 'n',
 'd',
 'o',
 'w',
 's',
 ' ',
 'i',
 's',
 ' ',
 'l',
 'e',
 's',
 's',
 ' ',
 't',
 'h',
 'a',
 'n',
 ' ',
 'i',
 'f',
 ' ',
 't',
 'h',
 'e',
 ' ',
 'w',
 'i',
 'n',
 'd',
 'o',
 'w',
 's',
 ' ',
 'a',
 'r',
 'e',
 ' ',
 'o',
 'p',
 'e',
 'n',
 ' ',
 't',
 'o',
 ' ',
 'c',
 'o',
 'o',
 'l',
 ' ',
 't',
 'h',
 'e',
 ' ',
 'o',
 'c',
 'c',
 'u',
 'p'

In [24]:
# identify the first and last token of the answer in the context or return no answer

# locate the start and end character of answer
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

The answer is not in this feature.


In [25]:
print(token_start_index)

10


In [27]:
# verify that the start and end tokens produced are the correct answer
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["text"][0])

NameError: ignored

In [51]:
# To make this notebook generalizable to any model, we account for the special case where the model expects padding on the left
pad_on_right = tokenizer.padding_side == "right"

In [52]:
# This function combines the above methods by tokenizing each example with truncation and padding

def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [53]:
# the function can work on multiple features. Verify that the tokenization is working correctly
features = prepare_train_features(datasets['train'][:2])
features

{'input_ids': [[101, 2043, 2106, 2250, 14372, 2150, 3115, 1999, 3765, 102, 9935, 2250, 14372, 1006, 2036, 2170, 1037, 1013, 1039, 1007, 3001, 2224, 2250, 14372, 2000, 4658, 1996, 2250, 1999, 1037, 4316, 1012, 1037, 2194, 1999, 2047, 2259, 2103, 1999, 1996, 2142, 2163, 2034, 3253, 8272, 1997, 2250, 14372, 2005, 3765, 1999, 4537, 1012, 2087, 1997, 2037, 6304, 3498, 28012, 2015, 1998, 9542, 3765, 1012, 1999, 3912, 1010, 24100, 2150, 1996, 2034, 9935, 7751, 2000, 3749, 2019, 2250, 14372, 3131, 1999, 2049, 3765, 1012, 2122, 2020, 7609, 2011, 3387, 1998, 8670, 9818, 7432, 2522, 1010, 1997, 6044, 1010, 4058, 1012, 1996, 1036, 1036, 3387, 1998, 8670, 9818, 7432, 4633, 4650, 2121, 1005, 1005, 2036, 5100, 1037, 3684, 2121, 1012, 3765, 3641, 2007, 1996, 2047, 1036, 1036, 4633, 4650, 2121, 1005, 1005, 2020, 12057, 2013, 24100, 1005, 1055, 2264, 2882, 8459, 4322, 2000, 1996, 1038, 1004, 1038, 4713, 2073, 1996, 7584, 2001, 2864, 1012, 2320, 3143, 1010, 1996, 2482, 2001, 12057, 2000, 1037, 2334, 1103

In [31]:
# apply the function on all elements of all the splits in the dataset including training, validation, and testing data
# remove the old columns since the preprocessing changes the number of samples
# results are cached. Pass "load_from_cache_file=False" to force the preprocessing to be applied again
tokenized_datasets = datasets.map(
    prepare_train_features, 
    batched=True, 
    remove_columns=datasets["train"].column_names
)

Fine-Tune the Model

In [32]:
# import Pytorch pretrained model for question answering
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

# from_pretrained method downloads and caches the model
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

# warning regarding not using weights and layers is normal. we are removing the 
# masked language modeling head to pretrain the model on the QA task for which
# we do not have pretrained weights and requires fine-tuning

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [33]:
########### INPUT ###########
# training arguments is a class that contains the attributes to customize training
# set the folder name f"model-dataset", which will be used to save checkpoints
# set the learning_rate, number of epochs, and weight_decay
# batch_size has been set at the beginning of the notebook 
args = TrainingArguments(
    f"bert-nq-qg",
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = 3,
    weight_decay = 0.01,
)

In [34]:
# import a default data collator
from transformers import default_data_collator

# set the data_collator to the default data collator
data_collator = default_data_collator

In [35]:
# pass all of the training arguments and datasets to the trainer
trainer = Trainer(
    model,
    args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"],
    data_collator = data_collator,
    tokenizer = tokenizer,
)

In [36]:
# finetune the model by calling train method
# running this cell will take time.
trainer.train()

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,0.487400,0.435194,204.085500,77.232000
2,0.332300,0.431457,204.042400,77.249000
3,0.215500,0.488653,203.693300,77.381000


TrainOutput(global_step=11655, training_loss=0.3767050469397271, metrics={'train_runtime': 8362.3106, 'train_samples_per_second': 1.394, 'total_flos': 4.67761639473239e+16, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 334895, 'init_mem_gpu_alloc_delta': 436709888, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1176410, 'train_mem_gpu_alloc_delta': 1308321792, 'train_mem_cpu_peaked_delta': 97198401, 'train_mem_gpu_peaked_delta': 8290021376})

In [37]:
########### INPUT ###########
# save the model. input the model name ("model-dataset-trained")
trainer.save_model("bert-nq-cl-reduce-trained")

Evaluation

In [38]:
# the validation features will need to be re-processed similar to the training features
# the processing will also need to check if the output span is inside the context (and not in the question)
# it will also need to retrieve the text inside

def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [39]:
# apply the function to validation set 
# remove the old columns since the preprocessing changes the number of samples
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)

In [40]:
# extract the predictions for all features using method trainer.predict
raw_predictions = trainer.predict(validation_features)

In [41]:
# trainer hides columns not used by the model. the columns needed for post-processing are set back 
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [42]:
########### INPUT ###########
# to classify answers, we use the score obtained by adding the start and end logits
# limit the number of possible answers by setting n_best_size
# limit the length of the answer by setting max_answer_length
n_best_size = 20
max_answer_length = 30

In [43]:
# get the output logits from trainer
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [44]:
# code to verify the score and corresponding text are working correctly
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 15.683541, 'text': 'W. Winter'},
 {'score': 10.421932, 'text': 'W. Winter -- known as Wint'},
 {'score': 9.84499, 'text': 'Winter'},
 {'score': 9.077557, 'text': 'W. Winter -- known as Wint -- and Charles Kidd'},
 {'score': 7.584131, 'text': '. Winter'},
 {'score': 7.092781, 'text': 'W'},
 {'score': 6.3153315, 'text': 'W. Winter -- known as Wint --'},
 {'score': 4.5833807, 'text': 'Winter -- known as Wint'},
 {'score': 4.355462, 'text': 'W. Winter --'},
 {'score': 3.8281991, 'text': 'W.'},
 {'score': 3.621228, 'text': 'Wint'},
 {'score': 3.2390056, 'text': 'Winter -- known as Wint -- and Charles Kidd'},
 {'score': 2.8396778, 'text': 'Bruce Glover'},
 {'score': 2.3225212, 'text': '. Winter -- known as Wint'},
 {'score': 2.276853, 'text': 'Wint -- and Charles Kidd'},
 {'score': 1.5263276,
  'text': 'Bruce Glover and Kidd by bespectacled jazz musician Putter Smith'},
 {'score': 1.0772647, 'text': 'Putter Smith'},
 {'score': 0.9781464, 'text': '. Winter -- known as Wint -- and C

In [45]:
# view the actual answer
datasets["validation"][0]["answers"]

{'answer_start': [631], 'text': ['bespectacled jazz musician Putter Smith']}

In [46]:
# apply the process above to all features by mapping between examples and their corresponding features
import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [47]:
# to handle the non-answerable questions, we need to extract the score for the impossible answer
# the score is collected from minimum of the scores from the CLS token for each feature generated by the example
# the question is not answerable when that score is greater than the highest answerable score
from tqdm.auto import tqdm
import numpy as np

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [48]:
# apply the postprocessing function to the raw predictions
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)

Post-processing 1964 example predictions split into 15762 features.


In [49]:
########### INPUT ###########
# load the metric from the datasets library
metric = load_metric("squad_v2")

In [50]:
# format predictions and labels
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

{'HasAns_exact': 18.930041152263374,
 'HasAns_f1': 21.948244579483777,
 'HasAns_total': 1458,
 'NoAns_exact': 85.7707509881423,
 'NoAns_f1': 85.7707509881423,
 'NoAns_total': 506,
 'best_exact': 36.15071283095723,
 'best_exact_thresh': 0.0,
 'best_f1': 38.391313949535274,
 'best_f1_thresh': 0.0,
 'exact': 36.15071283095723,
 'f1': 38.39131394953531,
 'total': 1964}